# Data Preparation

## Loading Prerequisites and Libraries 

**Standard Imports:**  
Plotly is configured to run in the offline mode.  
Pandas is chosen to for manipulating csv files.  
Numpy is for the arithmetic applications.

**Visualizations:**  
Most of the visualizations are done via matplotlib, seaborn and plotly.   

**Custom Functions:**  
All additional functions and code are written and imported from the **eda_xray.py** file.

In [25]:
%load_ext autoreload
%autoreload 2
import os
from eda_xray import load_xray
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  # sets up plotting under plt
import matplotlib.cm as cm  # allows us easy access to colormaps
import plotly.express as px
import plotly.graph_objs as go

%matplotlib inline

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('precision', 5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


There is a total of 112120 images on the NIH dataset.

In [26]:
# creating image location and looking for the number of the images
img_location = 'data/images'
len([file for file in os.listdir(img_location) if file.endswith('.png')])

112120

## Importing data and renaming/dropping columns

The function **load_xray** does the following:
1. imports the css
2. renames the columns to lower letter
3. drops the last empty column
4. adds a new columns caled **'path'** that has the x-rays *image*.png* location

In [27]:
data = 'data/Data_Entry_2017.csv'
df = load_xray(data)
df.sample(5)

,imgindex,label,followup,patientID,age,gender,viewposition,width,height,x,y,path
38115,00010007_034.png,Pleural_Thickening,34,10007,58,M,AP,2500,2048,0.168,0.168,data/images/00010007_034.png
107903,00029174_002.png,Atelectasis|Infiltration,2,29174,5,M,AP,2544,3056,0.139,0.139,data/images/00029174_002.png
19015,00005066_011.png,Cardiomegaly|Infiltration,11,5066,49,M,PA,2500,2048,0.168,0.168,data/images/00005066_011.png
60699,00014989_000.png,No Finding,0,14989,23,M,PA,2992,2991,0.143,0.143,data/images/00014989_000.png
90076,00022398_000.png,Pleural_Thickening,0,22398,26,M,PA,2992,2991,0.143,0.143,data/images/00022398_000.png


# Exploratory Data Analysis

## Unique pathologies of x-ray abnormalities and choosing a subset

There 836 unique values in the labels and these values have several different groupings of the pathologies. 

There are 836 different combinations of labeling among the x-rays.

In [28]:
df.label.nunique()

836

If we don't the consider the multilabel abnormalities, we'd have 14 labels plus the *'No Finding'* classification.

In [30]:
single_labels = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax',
                 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia',
                 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia']
print(df[df.label.isin(single_labels)].label.value_counts())

fig = px.bar(df[df.label.isin(single_labels)].label.value_counts().reset_index(),
       x='index',
       y='label',
       title='X-ray counts for single label pathologies',
       labels={'index': 'Pathology', 'label': 'Count'},
       template='plotly_white', width=800, height=600)
fig.update_traces(marker_color='rgb(128,128,255)', marker_opacity=0.8)
fig.show()

Infiltration          9547
Atelectasis           4215
Effusion              3955
Nodule                2705
Pneumothorax          2194
Mass                  2139
Consolidation         1310
Pleural_Thickening    1126
Cardiomegaly          1093
Emphysema              892
Fibrosis               727
Edema                  628
Pneumonia              322
Hernia                 110
Name: label, dtype: int64


## Removing  items with less than 20 images

In [36]:
df.label.value_counts()

No Finding                                                  60361
Infiltration                                                 9547
Atelectasis                                                  4215
Effusion                                                     3955
Nodule                                                       2705
                                                            ...  
Consolidation|Emphysema|Infiltration|Mass|Nodule                1
Hernia|Infiltration|Nodule                                      1
Edema|Effusion|Pleural_Thickening|Pneumonia                     1
Atelectasis|Edema|Infiltration|Nodule|Pleural_Thickening        1
Atelectasis|Hernia|Mass                                         1
Name: label, Length: 836, dtype: int64

By looking at the values of the pathology labels, we can see that for some labels/findings there are only one image and some have a few more. To facilitate classification and modeling we remove the x-rays with labels, that have less than 20 instances.

In [37]:
df = df.groupby('label').filter(lambda x : len(x)>20)

## Creating Categorical Variables

The **create_categorical** function creates dummy variables through pandas and creates a *'target'* column with a scalar vector of the dummy variables for later comparison of the convolutional neural networks results.

In [38]:
from eda_xray import create_categorical
df = create_categorical(df)
df.sample(5)

C:\Users\yasha\OneDrive\#DS\PROJECTS\capstone\eda_xray.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yasha\OneDrive\#DS\PROJECTS\capstone\eda_xray.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,imgindex,label,followup,patientID,age,gender,viewposition,width,height,x,y,path,Atelectasis,Consolidation,Infiltration,Pneumothorax,Edema,Emphysema,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia,target
110210,00029998_008.png,Atelectasis|Infiltration,8,29998,31,M,AP,3056,2544,0.139,0.139,data/images/00029998_008.png,1,0,1,0,0,0,0,0,0,0,0,0,0,0,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
98513,00026032_004.png,No Finding,4,26032,73,F,AP,3056,2544,0.139,0.139,data/images/00026032_004.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
40482,00010531_043.png,No Finding,43,10531,23,F,PA,2834,2991,0.143,0.143,data/images/00010531_043.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
25388,00006679_013.png,Edema,13,6679,22,F,AP,2500,2048,0.168,0.168,data/images/00006679_013.png,0,0,0,0,1,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
70935,00017504_013.png,No Finding,13,17504,12,F,AP,2048,2500,0.168,0.168,data/images/00017504_013.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


## Saving Subset To Pickle

In [39]:
import os
if not os.path.exists('subset'):
    os.makedirs('subset')

In [40]:
df.to_pickle('subset/subset.pkl')

## Split train, validation, and test data

In [43]:
from sklearn.model_selection import train_test_split

train_set, valid_set = train_test_split(df, test_size=0.3,
                                        random_state=42,
                                        stratify=df.label)

train_set, test_set = train_test_split(train_set, test_size=0.3,
                                       random_state=42,
                                       stratify=train_set.label)


print('training set values: ', train_set.shape[0])
print('validation set values: ', valid_set.shape[0])
print('testing set values: ', test_set.shape[0])
print('subset data values:', df.shape[0])

training set values:  53692
validation set values:  32874
testing set values:  23011
subset data values: 109577


## Visualizing the Split

In [46]:
# import plotly.graph_objects as go
# labels = train_set.label.value_counts().keys()
# train_counts = train_set.label.value_counts()
# valid_counts = valid_set.label.value_counts()
# test_counts = test_set.label.value_counts()
# fig = go.Figure(data=[
#     go.Bar(name='Training Set', x=labels,
#            y=train_set.label.value_counts()),
#     go.Bar(name='Validation Set', x=labels,
#            y=valid_set.label.value_counts()),
#     go.Bar(name='Testing Set', x=labels,
#            y=test_set.label.value_counts())
# ])
# # Change the bar mode
# fig.update_layout(barmode='stack', template = 'plotly_white', 
#                   title = 'Training, Validation and Testing Splits')
# fig.show()

## Saving Training, Validation and Testing Sets to Pickle

In [45]:
train_set.to_pickle('subset/train_set.csv')
valid_set.to_pickle('subset/valid_set.csv')
test_set.to_pickle('subset/test_set.csv')